# PUBG 크롤링

1. 게임 전체 리뷰
    - 수집 대상
        - 닉네임, 작성일, 본문, 추천 여부, 플레이 시간
    - 스크롤 내리면서 리뷰 한 개 섹션 단위로 파싱
    - 중복된 댓글 반복 시 종료
    - 500개마다 중간저장
    
2. 스팀 뉴스 업데이트 수집
    - 수집 대상
        - 업데이트 페이지에서 각 세부 페이지 경로 수집
        - 이후 제목, 게시일, 유형, 본문, 좋아요 수, 댓글 주소 수집

3. 인스타그램 수집
    - 수집 대상
        - 인스타그램 게시물 링크
        - 게시일, 좋아요 수, 본문, 게시물 URL
    - 로그인 후 프로필 페이지 진입
    - 스크롤 다운하며 링크 수집
    - 2024-07-01 이전 게시물 발견 시 루프 중단

4. 유튜브 수집
    - 수집 대상
        - 제목, 동영상 URL, 조회수 텍스트, 게시일
    - 링크 이동 후 '동영상' 탭 클릭
    - 페이지 최하단까지 스크롤 후 파싱
    - URL 하나씩 열어서 좋아요 수, 댓글 수 파싱
    - URL을 기준으로 맵핑하여 하나의 데이터프레임으로 완성 후 저장

## 1. 게임 전체 리뷰

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

# 셀레니움 설정
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options)

# URL 진입
url = "https://steamcommunity.com/app/578080/reviews/?p=1&browsefilter=trendthreemonths&filterLanguage=all"
driver.get(url)
time.sleep(3)

SCROLL_PAUSE = 2
results = []
last_height = driver.execute_script("return document.body.scrollHeight")
last_seen_content = None
seen_repeat_count = 0

save_interval = 500
save_count = 1

while True:
    # 리뷰 카드 수집
    cards = driver.find_elements(By.CLASS_NAME, "apphub_Card")

    for card in cards[len(results):]:
        try:
            nickname_link = card.find_element(By.CSS_SELECTOR, ".apphub_friend_block_container a").get_attribute("href")
        except:
            nickname_link = None

        try:
            date = card.find_element(By.CLASS_NAME, "date_posted").text.replace("게시 일시: ", "").strip()
        except:
            date = None

        try:
            content = card.find_element(By.CLASS_NAME, "apphub_CardTextContent").text.strip()
        except:
            content = None

        try:
            recommendation = card.find_element(By.CLASS_NAME, "title").text.strip()
        except:
            recommendation = None

        try:
            playtime = card.find_element(By.CLASS_NAME, "hours").text.strip()
        except:
            playtime = None

        # 종료 조건: 동일 댓글 반복 감지
        if content == last_seen_content:
            seen_repeat_count += 1
            if seen_repeat_count >= 3:
                print("\n동일 댓글 3회 반복 감지. 크롤링 종료.")
                break
        else:
            seen_repeat_count = 0
            last_seen_content = content

        row = {
            "닉네임": nickname_link,
            "작성일": date,
            "본문": content,
            "추천 여부": recommendation,
            "플레이 시간": playtime
        }
        results.append(row)

        # 실시간 출력
        print(f"\n닉네임: {nickname_link}")
        print(f"작성일: {date}")
        print(f"추천여부: {recommendation}")
        print(f"플레이 시간: {playtime}")
        print(f"본문: {content[:100]}{'...' if content and len(content) > 100 else ''}")

        # 중간 저장
        if len(results) % save_interval == 0:
            temp_df = pd.DataFrame(results)
            temp_df.to_csv(f"PUBG_reviews_temp_{save_count}.csv", index=False, encoding="utf-8-sig")
            print(f"\n 중간 저장 완료: PUBG_reviews_temp_{save_count}.csv ({len(results)}개)")
            save_count += 1

    # 중복 종료 처리
    if seen_repeat_count >= 3:
        break

    # 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)  # 스크롤 후 5초 대기

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("\n더 이상 로드할 내용 없음. 종료합니다.")
        break
    last_height = new_height


# 최종 저장
df = pd.DataFrame(results)
df.to_csv("32-0. PUBG_reviews.csv", index=False, encoding="utf-8-sig")
print(f"\n 크롤링 완료! 총 리뷰 수: {len(df)}개")
print("저장 완료: 32-0. PUBG_reviews.csv")

In [2]:
df = pd.read_csv('PUBG_reviews_temp_72.csv', encoding='utf-8-sig')

In [5]:
df.to_csv("32-0. PUBG_reviews.csv", index=False, encoding="utf-8-sig")

## 2. 스팀 뉴스 업데이트 수집

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# ▶ URL 설정
url = "https://store.steampowered.com/news/app/578080?updates=true&l=koreana"

# ▶ 드라이버 설정
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ▶ 페이지 열기
driver.get(url)
time.sleep(3)

# ▶ 수집용 set
collected_urls = set()
results = []

# ▶ 천천히 스크롤하며 수집
scroll_count = 0
max_scrolls = 50
no_new_count = 0

while scroll_count < max_scrolls and no_new_count < 3:
    soup = BeautifulSoup(driver.page_source, "html.parser")

    news_blocks = soup.find_all("div", class_="_398u23KF15gxmeH741ZSyL")

    found_this_round = 0

    for block in news_blocks:
        a_tag = block.find("a", class_="Focusable")
        if a_tag:
            href = a_tag.get("href")
            if href and href not in collected_urls:
                full_url = "https://store.steampowered.com" + href
                collected_urls.add(href)
                results.append({"링크": full_url})
                print(f"📝 [{len(results)}] {full_url}")
                found_this_round += 1

    if found_this_round == 0:
        no_new_count += 1
    else:
        no_new_count = 0

    # 스크롤 다운
    driver.execute_script("window.scrollBy(0, 1500);")
    scroll_count += 1
    time.sleep(2.5)

# ▶ 저장
updatelink_df = pd.DataFrame(results)
updatelink_df['링크'] = updatelink_df["링크"].str.extract(r"(\/view\/\d+)")
updatelink_df.to_csv("32-1. PUBG_links.csv", index=False, encoding="utf-8-sig")
print(f"\n✅ 총 {len(updatelink_df)}개 뉴스 링크 저장 완료 → 32-1. PUBG_links.csv")

driver.quit()

📝 [1] https://store.steampowered.com/news/app/578080/view/3883856311524787330
📝 [2] https://store.steampowered.com/news/app/578080/view/3881602608981008067
📝 [3] https://store.steampowered.com/news/app/578080/view/3727351145888357539
📝 [4] https://store.steampowered.com/news/app/578080/view/3737482343880471080
📝 [5] https://store.steampowered.com/news/app/578080/view/3677806476751507188
📝 [6] https://store.steampowered.com/news/app/578080/view/3690188838542955859
📝 [7] https://store.steampowered.com/news/app/578080/view/3692437468646527034
📝 [8] https://store.steampowered.com/news/app/578080/view/3686806065682225286
📝 [9] https://store.steampowered.com/news/app/578080/view/3719451457286586144
📝 [10] https://store.steampowered.com/news/app/578080/view/3666531626460999206
📝 [11] https://store.steampowered.com/news/app/578080/view/3681165789407271674
📝 [12] https://store.steampowered.com/news/app/578080/view/3681163252615600258
📝 [13] https://store.steampowered.com/news/app/578080/view/35

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# ▶ 드라이버 세팅
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ▶ URL 베이스
base_url = "https://store.steampowered.com/news/app/578080"

# ▶ update_df["링크"]에 '/view/숫자' 형태가 들어있다고 가정
# 예시용 (실제에는 update_df = pd.read_csv(...)로 불러와야 함)
# update_df = pd.read_csv("steam_news_links_stable.csv")
# update_df["링크"] = update_df["링크"].str.extract(r"(\/view\/\d+)")

results = []

for i, update_link in enumerate(updatelink_df["링크"], 1):
    full_url = base_url + update_link + "?l=koreana"
    print(f"\n🌐 [{i}] 크롤링 중: {full_url}")

    try:
        driver.get(full_url)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # ✅ 제목
        title_tag = soup.select_one("div._3z2NYCkFizMu4fMvWTIBUG div.TqEPC9bhvVpZ1rb3Z8Mbd")
        title = title_tag.text.strip() if title_tag else ""

        # ✅ 게시일
        date_tag = soup.select_one("div._3IxVZE9uydjh3cA9kmtnk7 div._2KsEbGy9kiSDeQpcqEc9DG div._1Maw_Rw6sOKYC1KkOI1xM")
        date = date_tag.text.strip() if date_tag else ""

        # ✅ 유형
        type_tag = soup.select_one("div._3phfIcOe_STA7hSoFfIxlE")
        type_text = type_tag.text.strip() if type_tag else ""

        # ✅ 본문
        body_tag = soup.select_one("div.EventDetailsBody.A_A2B6fTn_MPLlGCmsLtd")
        body = body_tag.text.strip() if body_tag else ""

        # ✅ 좋아요 수
        like_tag = soup.select_one("div._9x4Z7eMgdwfAVMr16ZaJ0 div._3Kelh1-_v6xHfRjF68n7NB div._3csl-MPe-hKuT8hQpOqEG5")
        likes = like_tag.text.strip() if like_tag else ""

        # ✅ 토론 주소
        comment_tag = soup.select_one("div._16xC0mtOWoLbvSQbmo_ycv a.Focusable")
        comment_link = comment_tag["href"] if comment_tag else ""

        # ▶ 저장
        results.append({
            "링크": full_url,
            "제목": title,
            "게시일": date,
            "유형": type_text,
            "본문": body,
            "좋아요": likes,
            "댓글 주소": comment_link
        })

        print(f"📝 제목: {title} | 게시일: {date} | 좋아요: {likes} | 본문: {body[:20]}")

    except Exception as e:
        print(f"❌ [{i}] 오류 발생: {e}")
        continue

driver.quit()

# ▶ DataFrame 저장
update_df = pd.DataFrame(results)
update_df.to_csv("32-2. PUBG_update_details.csv", index=False, encoding="utf-8-sig")
print(f"\n✅ 총 {len(update_df)}개 뉴스 상세 정보 저장 완료 → 32-2. PUBG_update_details.csv")



🌐 [1] 크롤링 중: https://store.steampowered.com/news/app/578080/view/3883856311524787330?l=koreana
📝 제목: 패치 노트 - 업데이트 27.2 | 게시일: 게시 일시 2024년 1월 10일 수 -오후 3:00 KST | 좋아요: 3,891 | 본문: 27.2 하이라이트라이브 점검 일정※

🌐 [2] 크롤링 중: https://store.steampowered.com/news/app/578080/view/3881602608981008067?l=koreana
📝 제목: Patch Notes - Update 27.1 | 게시일: 게시 일시 2023년 12월 6일 수 -오후 3:00 KST | 좋아요: 2,700 | 본문: 27.1 HighlightsLive 

🌐 [3] 크롤링 중: https://store.steampowered.com/news/app/578080/view/3727351145888357539?l=koreana
📝 제목: Patch Notes - Update 26.2 | 게시일: 게시 일시 2023년 11월 1일 수 -오후 3:00 KST | 좋아요: 2,530 | 본문: 26.2 HighlightsLive 

🌐 [4] 크롤링 중: https://store.steampowered.com/news/app/578080/view/3737482343880471080?l=koreana
📝 제목: 패치 노트 - 업데이트 26.1 | 게시일: 게시 일시 2023년 10월 5일 목 -오후 3:00 KST | 좋아요: 4,680 | 본문: ﻿26.1 하이라이트﻿라이브 점검 일

🌐 [5] 크롤링 중: https://store.steampowered.com/news/app/578080/view/3677806476751507188?l=koreana
📝 제목: 패치 노트 - 업데이트 25.2 | 게시일: 게시 일시 2023년 8월 30일 수 -오후 3:00 KST | 좋아요: 2,086 | 본문: 

## 3. 인스타그램 수집

In [11]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys 
import time 
import pandas as pd 
from datetime import datetime 
from bs4 import BeautifulSoup 

# === 1. 사용자 계정 정보 === 
INSTAGRAM_ID = "ddukbbangii_i"
INSTAGRAM_PW = "qawsed123" 

# === 2. Chrome 드라이버 설정 === 
options = Options() 
options.add_argument("--start-maximized") 
driver = webdriver.Chrome(options=options) 

# === 3. 인스타그램 로그인 페이지로 이동 === 
driver.get("https://www.instagram.com/") 
time.sleep(5) 

# === 4. 로그인 === 
try: 
    id_input = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div[1]/div[1]/div/label/input') 
    id_input.send_keys(INSTAGRAM_ID) 
    time.sleep(1) 
    pw_input = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div[1]/div[2]/div/label/input') 
    pw_input.send_keys(INSTAGRAM_PW) 
    time.sleep(1) 
    login_btn = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div[1]/div[3]/button') 
    login_btn.click() 
    print("🔐 로그인 시도 중...") 
    time.sleep(7) 
except Exception as e: 
    print(f"❌ 로그인 실패: {e}") 

# === 5. 타겟 페이지 이동 === 
target_url = "https://www.instagram.com/pubg_battlegrounds_kr/" 
driver.get(target_url) 
time.sleep(5) 

# === 6. 게시물 URL 수집 함수 === 
def collect_post_urls(max_posts=200): 
    """스크롤하면서 게시물 URL들을 수집""" 
    print(f"📜 최대 {max_posts}개 게시물 URL 수집 중...") 
    collected_urls = set() # 중복 방지 
    last_height = driver.execute_script("return document.body.scrollHeight") 
    no_new_posts_count = 0 
    
    while len(collected_urls) < max_posts and no_new_posts_count < 3: 
        # 현재 페이지에 있는 모든 게시물 링크 찾기 
        try: 
            # 여러 선택자로 시도 
            selectors = [ 
                "a.x1i10hfl[href*='/p/']", 
                "a[href*='/p/']", 
            ] 
            current_posts = [] 
            for selector in selectors: 
                try: 
                    posts = driver.find_elements(By.CSS_SELECTOR, selector) 
                    if posts: 
                        current_posts = posts 
                        break 
                except: 
                    continue 

            # URL 수집 
            before_count = len(collected_urls) 
            for post in current_posts: 
                try: 
                    href = post.get_attribute("href") 
                    if href and ("/p/" in href or "/reel/" in href): 
                        collected_urls.add(href) 
                except: 
                    continue 

            after_count = len(collected_urls) 
            new_found = after_count - before_count 
            print(f"📊 현재까지 수집된 URL: {len(collected_urls)}개 (새로 발견: {new_found}개)") 

            if new_found == 0: 
                no_new_posts_count += 1 
                print(f"⚠️ 새로운 게시물 없음 ({no_new_posts_count}/3)") 
            else: 
                no_new_posts_count = 0 

            # 스크롤 
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
            time.sleep(4) 

            # 페이지 높이 확인 
            new_height = driver.execute_script("return document.body.scrollHeight") 
            if new_height == last_height: 
                no_new_posts_count += 1 
                print(f"⚠️ 페이지 높이 변화 없음 ({no_new_posts_count}/3)") 
            else: 
                last_height = new_height 
                
        except Exception as e: 
            print(f"❌ URL 수집 중 오류: {e}") 
            break 

    print(f"✅ URL 수집 완료: 총 {len(collected_urls)}개") 
    return list(collected_urls) 

# === 7. 게시물 URL 수집 === 
post_urls = collect_post_urls(max_posts=100) 
if not post_urls: 
    print("❌ 게시물 URL을 찾을 수 없습니다.") 
    driver.quit() 
    exit() 

# === 8. 각 게시물 방문하여 데이터 수집 === 
results = [] 
stop_date = datetime.strptime("2024.07.01", "%Y.%m.%d") 
print(f"\n🔄 {len(post_urls)}개 게시물 데이터 수집 시작...") 

for i, post_url in enumerate(post_urls, 1): 
    try: 
        print(f"📸 [{i}/{len(post_urls)}] 게시물 처리 중: {post_url}") 
        
        # 게시물 페이지로 직접 이동 
        driver.get(post_url) 
        time.sleep(5) 

        # 본문 수집 (실제 게시물 페이지 구조에 맞게 수정)
        content = ""
        try:
            print(f"🔍 본문 수집 시도 중...")
            
            # 실제 게시물 페이지의 구조에 맞는 선택자들
            content_selectors = [
                # 제공해주신 실제 구조
                "div.html-div.xdj266r.x14z9mp.xat24cr.x1lziwak.xexx8yu.xyri2b.x18d9i69.x1c1uobl.x9f619.xjbqb8w.x78zum5.x15mokao.x1ga7v0g.x16uus16.xbiv7yw.x1uhb9sk.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.xpm28yp.x8viiok.x1o7cslx.x126k92a",
                
                # 간단한 버전들
                "div.html-div span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xt0psk2",
                "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.xpm28yp.x8viiok.x1o7cslx.x126k92a",
                "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xt0psk2",
                "span.xt0psk2.x193iq5w",
                
                # 주요 클래스만으로
                "span.xt0psk2",
                "span.x193iq5w",
                "span.xeuugli.xt0psk2",
                
                # div.html-div 내부 모든 span
                "div.html-div span",
                "div.xdj266r span",
                
                # 더 광범위한 검색
                "div[class*='html-div'] span[class*='xt0psk2']",
                "div[class*='xdj266r'] span[class*='x193iq5w']",
            ]
            
            for selector in content_selectors:
                try:
                    print(f"  CSS 시도: {selector}")
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    print(f"    발견된 요소 수: {len(elements)}개")
                    
                    for i, element in enumerate(elements):
                        text = element.text.strip()
                        print(f"    [{i+1}] 텍스트: {text[:80]}...")
                        
                        # 본문 조건 체크
                        if (text and 
                            len(text) > 10 and 
                            text != "..." and
                            "좋아요" not in text and 
                            "like" not in text.lower() and
                            "팔로우" not in text and 
                            "follow" not in text.lower() and
                            "댓글" not in text and 
                            "comment" not in text.lower() and
                            "공유" not in text and 
                            "저장" not in text and
                            "더 보기" not in text and
                            not text.replace(",", "").replace(".", "").replace("만", "").isdigit()):
                            
                            content = text
                            print(f"✅ 본문 발견! 선택자: {selector}")
                            print(f"   길이: {len(text)}자, 내용: {text[:100]}...")
                            break
                    
                    if content:
                        break
                        
                except Exception as e:
                    print(f"    CSS 실패: {e}")
                    continue
            
            # CSS로 못 찾으면 XPath로 시도
            if not content:
                print("🔍 XPath로 재시도...")
                xpath_selectors = [
                    # 클래스 기반 XPath
                    "//div[contains(@class, 'html-div')]//span[contains(@class, 'xt0psk2')]",
                    "//div[contains(@class, 'xdj266r')]//span[contains(@class, 'x193iq5w')]",
                    "//span[contains(@class, 'xt0psk2') and contains(@class, 'x193iq5w')]",
                    "//span[contains(@class, 'xt0psk2')]",
                    "//span[contains(@class, 'x193iq5w')]",
                    
                    # 긴 텍스트가 있는 span
                    "//span[string-length(text()) > 15]",
                    "//div[contains(@class, 'html-div')]//span[string-length(text()) > 15]",
                ]
                
                for xpath in xpath_selectors:
                    try:
                        print(f"  XPath 시도: {xpath}")
                        elements = driver.find_elements(By.XPATH, xpath)
                        print(f"    발견된 요소 수: {len(elements)}개")
                        
                        for i, element in enumerate(elements):
                            text = element.text.strip()
                            print(f"    [{i+1}] 텍스트: {text[:80]}...")
                            
                            if (text and 
                                len(text) > 15 and 
                                text != "..." and
                                "좋아요" not in text and 
                                "팔로우" not in text and
                                "댓글" not in text and
                                not text.replace(",", "").isdigit()):
                                
                                content = text
                                print(f"✅ XPath로 본문 발견!")
                                break
                        
                        if content:
                            break
                            
                    except Exception as e:
                        print(f"    XPath 실패: {e}")
                        continue
            
            # 마지막: 모든 span 요소 확인
            if not content:
                print("🔍 모든 span 요소 확인...")
                try:
                    all_spans = driver.find_elements(By.TAG_NAME, "span")
                    print(f"    총 {len(all_spans)}개의 span 요소 발견")
                    
                    for i, span in enumerate(all_spans):
                        try:
                            text = span.text.strip()
                            classes = span.get_attribute("class") or ""
                            
                            # 본문으로 보이는 긴 텍스트
                            if (text and 
                                len(text) > 20 and 
                                len(text) < 1000 and
                                text != "..." and
                                "좋아요" not in text and 
                                "팔로우" not in text and
                                "댓글" not in text and
                                ("xt0psk2" in classes or "x193iq5w" in classes or 
                                 "Step" in text or "Destiny" in text or "Nine" in text)):
                                
                                content = text
                                print(f"✅ span 탐색으로 본문 발견! 인덱스: {i+1}")
                                print(f"   클래스: {classes}")
                                print(f"   텍스트: {text[:100]}...")
                                break
                                
                        except Exception as e:
                            continue
                            
                except Exception as e:
                    print(f"  모든 span 확인 실패: {e}")
                    
        except Exception as e:
            print(f"❌ 본문 수집 전체 실패: {e}")
            content = ""
                    
        except Exception as e:
            print(f"⚠️ 본문 수집 실패: {e}")
            content = ""

        # 좋아요 수 수집 
        likes = "" 
        try: 
            # 여러 방법으로 좋아요 수 찾기 
            like_selectors = [ 
                "a[href*='liked_by']", 
                "span:contains('좋아요')", 
                "button span:contains('좋아요')", 
            ] 
            for selector in like_selectors: 
                try: 
                    if "contains" in selector: 
                        # XPath 사용 
                        xpath = f"//span[contains(text(), '좋아요')]" 
                        like_elem = driver.find_element(By.XPATH, xpath) 
                    else: 
                        like_elem = driver.find_element(By.CSS_SELECTOR, selector) 
                    likes = like_elem.text 
                    break 
                except: 
                    continue 

            # 좋아요 수를 찾지 못했을 때 숫자가 있는 span 찾기 
            if not likes: 
                try: 
                    spans = driver.find_elements(By.CSS_SELECTOR, "span") 
                    for span in spans: 
                        span_text = span.text.strip() 
                        if span_text and any(char.isdigit() for char in span_text): 
                            if "좋아요" in span_text or "like" in span_text.lower() or span_text.replace(',', '').replace('.', '').replace('만', '').isdigit(): 
                                likes = span_text 
                                break 
                except: 
                    pass 
        except: 
            likes = "" 

        # 게시일 수집 
        date_obj = None 
        try: 
            time_tag = WebDriverWait(driver, 5).until( 
                EC.presence_of_element_located((By.CSS_SELECTOR, "time")) 
            ) 
            date_text = time_tag.get_attribute("title") 
            if date_text: 
                # 한국어 날짜 형식 처리 
                date_text = date_text.replace("년 ", "-").replace("월 ", "-").replace("일", "") 
                date_obj = datetime.strptime(date_text, "%Y-%m-%d") 
                
                # 기준일 이전이면 종료 
                if date_obj < stop_date: 
                    print(f"🛑 기준일({stop_date.date()}) 이전 게시물 발견 → 종료") 
                    break 
        except Exception as e: 
            print(f"⚠️ 날짜 파싱 실패: {e}") 
            date_obj = None 

        print(f"✅ [{i}] 게시일: {date_obj.date() if date_obj else ''} | 좋아요: {likes} | 본문: {content[:30]}...") 
        
        results.append({ 
            "게시일": date_obj.date() if date_obj else "", 
            "좋아요 수": likes, 
            "본문": content, 
            "주소": post_url 
        }) 

        # 10개마다 진행 상황 출력 
        if i % 10 == 0: 
            print(f"🔄 {i}개 수집 완료") 
            
    except Exception as e: 
        print(f"⚠️ 게시물 {i} 처리 실패: {e}") 
        continue 

# === 9. 결과 저장 ===
try:
    insta_df = pd.DataFrame(results)
    insta_df.to_csv("32-3. PUBG_instagram_crawled.csv", index=False, encoding="utf-8-sig")
    print(f"\n✅ 저장 완료: 32-3. PUBG_instagram_crawled.csv (총 {len(insta_df)}개)")
except Exception as e:
    print(f"❌ 저장 실패: {e}")
finally:
    driver.quit()

🔐 로그인 시도 중...
📜 최대 100개 게시물 URL 수집 중...
📊 현재까지 수집된 URL: 3개 (새로 발견: 3개)
📊 현재까지 수집된 URL: 12개 (새로 발견: 9개)
📊 현재까지 수집된 URL: 17개 (새로 발견: 5개)
📊 현재까지 수집된 URL: 24개 (새로 발견: 7개)
📊 현재까지 수집된 URL: 30개 (새로 발견: 6개)
📊 현재까지 수집된 URL: 40개 (새로 발견: 10개)
📊 현재까지 수집된 URL: 43개 (새로 발견: 3개)
📊 현재까지 수집된 URL: 47개 (새로 발견: 4개)
📊 현재까지 수집된 URL: 50개 (새로 발견: 3개)
📊 현재까지 수집된 URL: 56개 (새로 발견: 6개)
📊 현재까지 수집된 URL: 62개 (새로 발견: 6개)
📊 현재까지 수집된 URL: 67개 (새로 발견: 5개)
📊 현재까지 수집된 URL: 74개 (새로 발견: 7개)
📊 현재까지 수집된 URL: 82개 (새로 발견: 8개)
📊 현재까지 수집된 URL: 89개 (새로 발견: 7개)
📊 현재까지 수집된 URL: 96개 (새로 발견: 7개)
📊 현재까지 수집된 URL: 101개 (새로 발견: 5개)
✅ URL 수집 완료: 총 101개

🔄 101개 게시물 데이터 수집 시작...
📸 [1/101] 게시물 처리 중: https://www.instagram.com/pubgesports_kr/p/DOVfMv7kxLm/
🔍 본문 수집 시도 중...
  CSS 시도: div.html-div.xdj266r.x14z9mp.xat24cr.x1lziwak.xexx8yu.xyri2b.x18d9i69.x1c1uobl.x9f619.xjbqb8w.x78zum5.x15mokao.x1ga7v0g.x16uus16.xbiv7yw.x1uhb9sk.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 span.x193iq5w.xeuugli.x13faqbe.x1vvk

## 4. 유튜브 수집

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

# --- 1. 크롬 드라이버 설정 ---
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

# --- 2. 유튜브 채널 열기 ---
driver.get("https://www.youtube.com/@PUBG_KR/videos")
time.sleep(3)

# --- 4. 영상 수집 ---
video_data = []
scroll_pause = 2
last_height = driver.execute_script("return document.documentElement.scrollHeight")
stop_crawling = False

while not stop_crawling:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    video_items = soup.select("ytd-rich-item-renderer")

    for item in video_items[len(video_data):]:  # 중복 방지
        title_tag = item.select_one("#video-title")
        link_tag = item.select_one("a#video-title-link")
        meta_tags = item.select("span.inline-metadata-item")

        if title_tag and link_tag and len(meta_tags) > 1:
            title = title_tag.text.strip()
            url = "https://www.youtube.com" + link_tag["href"]
            posted = meta_tags[1].text.strip()
            views = meta_tags[0].text.strip()

            # 종료 조건: 게시일이 '1년 전'
            if "1년 전" in posted:
                print(f"🛑 게시일 '{posted}' → 크롤링 종료")
                stop_crawling = True
                break

            print(f"📹 {title} | {url} | {views} | {posted}")
            video_data.append({
                "제목": title,
                "URL": url,
                "조회수": views,
                "게시일": posted
            })

    # 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(scroll_pause)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# --- 5. 브라우저 종료 ---
driver.quit()

# --- 6. CSV로 저장 ---
youtube_df = pd.DataFrame(video_data)
csv_path = "32-4. PUBG_youtube_crawled.csv"
youtube_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"\n✅ 총 {len(youtube_df)}개 영상 저장 완료: {csv_path}")

📹 미라마에 도달한 굶주린 자들 | 배틀그라운드 | 배그 | https://www.youtube.com/watch?v=Dp2c_RjbR5o | 조회수 5.7천회 | 1일 전
📹 PUBG x G-DRAGON 메인 트레일러 | PUBG 콜라보레이션 | https://www.youtube.com/watch?v=2hrX3MFvCmY | 조회수 67만회 | 11일 전
📹 [37.2 패치노트 미리보기] PUBG x G-DRAGON 콜라보, 신규 상호작용, 미라마 월드 업데이트 | 배틀그라운드 | 배그 | https://www.youtube.com/watch?v=QytY1ydC5_w | 조회수 12만회 | 12일 전
📹 PUBG x G-DRAGON, 그가 온다 | https://www.youtube.com/watch?v=y7c1_eFYK44 | 조회수 13만회 | 3주 전
📹 클랜컵 2025 SUMMER PARTY 현장 스케치 | 배틀그라운드 | 배그 | https://www.youtube.com/watch?v=EcbBKZuUaA0 | 조회수 1.6천회 | 3주 전
📹 자, 이제 다음은? | https://www.youtube.com/watch?v=w_BkjCc09yg | 조회수 2.3만회 | 4주 전
📹 돌아온 POBG (feat. 캡틴 크리스피) | 배틀그라운드 | 배그 | https://www.youtube.com/watch?v=Wf3YDGp5IXw | 조회수 4.4천회 | 1개월 전
📹 PUBG x BUGATTI 공식 트레일러 | 배틀그라운드 | 배그 | https://www.youtube.com/watch?v=4UJPx3hxcxI | 조회수 4.1만회 | 1개월 전
📹 [37.1 패치노트 미리보기] PUBG x BUGATTI 콜라보, DMR 리밸런스, POBG, UGC 알파 | 배틀그라운드 | 배그 | https://www.youtube.com/watch?v=tjvQexk-a7w | 조회수 11만회 | 1개월 전
📹 UGC 알파 가이드 5: 샘플 모드 제작 | 배

In [13]:
youtube_df = pd.read_csv("32-4. PUBG_youtube_crawled.csv", encoding="utf-8-sig")
yturl_list = youtube_df['URL'].tolist()

In [14]:
yturl_list

['https://www.youtube.com/watch?v=2hrX3MFvCmY',
 'https://www.youtube.com/watch?v=QytY1ydC5_w',
 'https://www.youtube.com/watch?v=y7c1_eFYK44',
 'https://www.youtube.com/watch?v=EcbBKZuUaA0',
 'https://www.youtube.com/watch?v=w_BkjCc09yg',
 'https://www.youtube.com/watch?v=Wf3YDGp5IXw',
 'https://www.youtube.com/watch?v=4UJPx3hxcxI',
 'https://www.youtube.com/watch?v=tjvQexk-a7w',
 'https://www.youtube.com/watch?v=QivCoLZipx4',
 'https://www.youtube.com/watch?v=Lm-7eUNhUsM',
 'https://www.youtube.com/watch?v=O_rBrisfPYw',
 'https://www.youtube.com/watch?v=qOPqGiZPLIo',
 'https://www.youtube.com/watch?v=oDlJX5BUHLE',
 'https://www.youtube.com/watch?v=s1gXIyle3f0',
 'https://www.youtube.com/watch?v=_uBw4qyh1A4',
 'https://www.youtube.com/watch?v=8kG-878v5l4',
 'https://www.youtube.com/watch?v=SvZcQAPRT7g&pp=0gcJCckJAYcqIYzv',
 'https://www.youtube.com/watch?v=GukSG9Z3IG4',
 'https://www.youtube.com/watch?v=8tbftbFbRj4&pp=0gcJCckJAYcqIYzv',
 'https://www.youtube.com/watch?v=dxfDCev5tJg',


In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re, time
import pandas as pd

# --- 드라이버 준비 ---
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# --- 숫자 변환 함수 ---
def to_int(text):
    if not text:
        return None
    text = text.upper()
    multiplier = 1
    if 'K' in text:
        multiplier = 1000; text = text.replace('K', '')
    elif 'M' in text:
        multiplier = 1000000; text = text.replace('M', '')
    elif '만' in text:
        multiplier = 10000; text = text.replace('만', '')
    elif '억' in text:
        multiplier = 100000000; text = text.replace('억', '')
    m = re.search(r'([\d,\.]+)', text.replace('\u00a0', ' '))
    if not m: return None
    try:
        return int(float(m.group(1).replace(',', '')) * multiplier)
    except:
        return None

# --- 댓글 수 수집 ---
def get_comment_count(driver):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.3);")
        time.sleep(2)
        WebDriverWait(driver, 8).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ytd-comments-header-renderer"))
        )
        selectors = [
            "ytd-comments-header-renderer h2#count yt-formatted-string span[dir='auto']",
            "ytd-comments-header-renderer h2#count yt-formatted-string span",
            "ytd-comments-header-renderer #count yt-formatted-string",
            "ytd-comments-header-renderer h2#count span",
            "ytd-comments-header-renderer .count-text span",
            "ytd-comments-header-renderer .count-text",
        ]
        for selector in selectors:
            try:
                text = driver.find_element(By.CSS_SELECTOR, selector).text.strip()
                count = to_int(text)
                if count is not None: 
                    return count
            except: 
                continue
    except:
        pass
    return 0

# --- 크롤링 ---
like_counts, comment_counts, clean_urls = [], [], []

for i, url in enumerate(yturl_list, 1):  # yturl_list는 미리 정의되어 있어야 함
    like_val, cmt_val = None, None
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "ytd-app"))
        )
        time.sleep(2)

        # 좋아요
        try:
            btns = driver.find_elements(By.XPATH, "//div[@id='top-level-buttons-computed']//button[@aria-label]")
            for b in btns:
                label = (b.get_attribute("aria-label") or "").lower()
                if any(k in label for k in ["좋아", "like"]):
                    like_val = to_int(label)
                    if like_val is not None:
                        break
        except: pass
        if like_val is None:
            try:
                el = driver.find_element(By.XPATH,
                    "//ytd-segmented-like-dislike-button-renderer//yt-smartimation//span[@id='text' or @id='button']"
                )
                like_val = to_int(el.text)
            except: pass

        # 댓글
        cmt_val = get_comment_count(driver)

    except:
        like_val, cmt_val = None, 0

    clean_urls.append(url)
    like_counts.append(like_val)
    comment_counts.append(cmt_val)

    # ✅ 결과만 출력
    print(f"[{i}/{len(yturl_list)}] {url}, 좋아요 수: {like_val}, 댓글 수: {cmt_val}")

driver.quit()

# --- DataFrame 저장 ---
yt_meta = pd.DataFrame({
    "URL": clean_urls,
    "좋아요 수": like_counts,
    "댓글 수": comment_counts
})

base_df = pd.read_csv("32-4. PUBG_youtube_crawled.csv", encoding="utf-8-sig")
enriched = base_df.merge(yt_meta, on="URL", how="left")
enriched.to_csv("32-5. PUBG_youtube_crawled_complete.csv", index=False, encoding="utf-8-sig")
print("✅ 저장 완료: 32-5. PUBG_youtube_crawled_complete.csv")

[1/94] https://www.youtube.com/watch?v=2hrX3MFvCmY, 좋아요 수: 2861, 댓글 수: 374
[2/94] https://www.youtube.com/watch?v=QytY1ydC5_w, 좋아요 수: 1303, 댓글 수: 441
[3/94] https://www.youtube.com/watch?v=y7c1_eFYK44, 좋아요 수: 2621, 댓글 수: 566
[4/94] https://www.youtube.com/watch?v=EcbBKZuUaA0, 좋아요 수: 33, 댓글 수: 4
[5/94] https://www.youtube.com/watch?v=w_BkjCc09yg, 좋아요 수: 179, 댓글 수: 159
[6/94] https://www.youtube.com/watch?v=Wf3YDGp5IXw, 좋아요 수: 55, 댓글 수: 20
[7/94] https://www.youtube.com/watch?v=4UJPx3hxcxI, 좋아요 수: 480, 댓글 수: 197
[8/94] https://www.youtube.com/watch?v=tjvQexk-a7w, 좋아요 수: 1300, 댓글 수: 872
[9/94] https://www.youtube.com/watch?v=QivCoLZipx4, 좋아요 수: 47, 댓글 수: 19
[10/94] https://www.youtube.com/watch?v=Lm-7eUNhUsM, 좋아요 수: 58, 댓글 수: 23
[11/94] https://www.youtube.com/watch?v=O_rBrisfPYw, 좋아요 수: 62, 댓글 수: 11
[12/94] https://www.youtube.com/watch?v=qOPqGiZPLIo, 좋아요 수: 71, 댓글 수: 12
[13/94] https://www.youtube.com/watch?v=oDlJX5BUHLE, 좋아요 수: 124, 댓글 수: 31
[14/94] https://www.youtube.com/watch?v=s1gX